## Ejercicio 5
Considerando el pagelink_sample.csv representar como un grafo en spark los contenidos de Wikipedia (considerando los contenidos como nodos y los links como aristas) como una lista de adyacencia y mostrar un recorrido en la estructura.

## Preparando sesión de Spark

### Spark session y Spark context

In [1]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Lectura de archivo

In [6]:
#import pandas as pd
import numpy as np 

In [3]:
#texts = pd.read_csv('./WikipediaDataset/pagelinks_sample.csv', usecols = ["pl_from", "pl_title"])
#texts2 = pd.read_csv('./WikipediaDataset/contents.csv', usecols = ["title"])
#texts = texts.loc[texts['pl_title'].isin(texts2['title']), :]
#texts.to_parquet('./WikipediaDataset/pagelinks_sample.parquet')
#del texts
#Dataset de pagelinks_sample.parquet fue creado con las líneas anteriores, y ahora puede ser leído
#de la siguiente forma:
dataframePageLinks = sqlContext.read.parquet('./WikipediaDataset/pagelinks_sample.parquet')
rddPageLinks = dataframePageLinks.rdd

In [7]:
#texts = pd.read_csv('./WikipediaDataset/contents.csv', usecols = ["title", "id"])
#texts.to_parquet('./WikipediaDataset/contentsTitleId.parquet')
#del texts
#Dataset de contentsTitleId.parquet fue creado con las líneas anteriores, y ahora puede ser leído
#de la siguiente forma:
dataframeContents = sqlContext.read.parquet('./WikipediaDataset/contentsTitleId.parquet')
rddContents = dataframeContents.rdd

## Preparando y limpiando los datasets

In [9]:
rddPageLinks1 = rddPageLinks
rddPageLinks1.take(5)

[Row(pl_from=6418187, pl_title='Insecta', __index_level_0__=2),
 Row(pl_from=6571966, pl_title='IMDb', __index_level_0__=8),
 Row(pl_from=5461523, pl_title='Vandalismo', __index_level_0__=12),
 Row(pl_from=1098340, pl_title='Wikidata', __index_level_0__=13),
 Row(pl_from=6224910, pl_title='Referencias', __index_level_0__=16)]

In [10]:
rddPageLinks1.count()

20572932

In [11]:
rddPageLinks1 = rddPageLinks1.map(lambda x: (x[0], x[1])).filter(lambda x: ((x[0] != None) and (x[1] != None)))
rddPageLinks1.take(5)

[(6418187, 'Insecta'),
 (6571966, 'IMDb'),
 (5461523, 'Vandalismo'),
 (1098340, 'Wikidata'),
 (6224910, 'Referencias')]

In [12]:
rddPageLinks1.count()

20572828

In [13]:
rddContents1 = rddContents

In [14]:
rddContents1.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5),
 Row(title='Andorra', id=7),
 Row(title='Argentina', id=10),
 Row(title='Geografía de Andorra', id=15),
 Row(title='Demografía de Andorra', id=17)]

In [15]:
rddContents1.count()

4132164

In [16]:
rddContents1 = rddContents1.filter(lambda x: ((x[0] != None) and (x[1] != None))).map(lambda x: (x[1], x[0]))
rddContents1.take(5)

[(5, 'Wikipedia:Artículos solicitados'),
 (7, 'Andorra'),
 (10, 'Argentina'),
 (15, 'Geografía de Andorra'),
 (17, 'Demografía de Andorra')]

In [17]:
rddContents1.count()

4132160

## Desarrollo

In [18]:
rddJoin = rddContents1.join(rddPageLinks1)
rddJoin.take(5)

[(2432, ('Wikipedia:Qué es un wiki', 'Wiki')),
 (20144, ('1358', 'Duque')),
 (20144, ('1358', '1358')),
 (20144, ('1358', '1359')),
 (20144, ('1358', 'Francia'))]

In [19]:
rddPageLinksConTitulos = rddJoin.map(lambda x: (x[1][0], x[1][1]))
rddPageLinksConTitulos.take(5)

[('Seat (estrella)', 'Wikidata'),
 ('Seat (estrella)', 'Sadachbia'),
 ('Seat (estrella)', 'Luminosidad'),
 ('Seat (estrella)', 'Kelvin'),
 ('Seat (estrella)', 'Km/s')]

In [22]:
rddListaDeAdyacencia = rddPageLinksConTitulos.groupBy(lambda x: x[0])
rddListaDeAdyacencia.take(5)

[('Luis Emilio Sierra',
  <pyspark.resultiterable.ResultIterable at 0x7f63986617c0>),
 ('HMS Curacoa (D41)',
  <pyspark.resultiterable.ResultIterable at 0x7f63986616d0>),
 ('Flakstadøya', <pyspark.resultiterable.ResultIterable at 0x7f6398661880>),
 ('Wikiproyecto:Mujeres/Mujeres en la arquitectura/Fotos e imágenes',
  <pyspark.resultiterable.ResultIterable at 0x7f6398661850>),
 ('Naftalina', <pyspark.resultiterable.ResultIterable at 0x7f6398661970>)]

In [23]:
rddListaDeAdyacencia.count()

1884622

In [28]:
rddListaDeAdyacencia = rddListaDeAdyacencia.map(lambda x: (x[0], list(x[1])))
rddListaDeAdyacencia.take(5)

[('Luis Emilio Sierra',
  [('Luis Emilio Sierra', '2006'),
   ('Luis Emilio Sierra', 'Manizales'),
   ('Luis Emilio Sierra', '2010'),
   ('Luis Emilio Sierra', '2014'),
   ('Luis Emilio Sierra', 'Abogado'),
   ('Luis Emilio Sierra', '2002'),
   ('Luis Emilio Sierra', '1960')]),
 ('HMS Curacoa (D41)',
  [('HMS Curacoa (D41)', 'Eslora'),
   ('HMS Curacoa (D41)', 'Derry'),
   ('HMS Curacoa (D41)', 'Km'),
   ('HMS Curacoa (D41)', 'Chatham'),
   ('HMS Curacoa (D41)', 'Mesana'),
   ('HMS Curacoa (D41)', 'Roda'),
   ('HMS Curacoa (D41)', 'Galerna'),
   ('HMS Curacoa (D41)', 'Greenock'),
   ('HMS Curacoa (D41)', 'Curacao'),
   ('HMS Curacoa (D41)', '1917'),
   ('HMS Curacoa (D41)', 'Proa'),
   ('HMS Curacoa (D41)', 'Dragaminas'),
   ('HMS Curacoa (D41)', 'Popa'),
   ('HMS Curacoa (D41)', '1916'),
   ('HMS Curacoa (D41)', 'Wikidata')]),
 ('Flakstadøya',
  [('Flakstadøya', 'OpenStreetMap'),
   ('Flakstadøya', 'Lofoten'),
   ('Flakstadøya', 'Noruega'),
   ('Flakstadøya', 'Flakstad'),
   ('Flaksta

In [78]:
def dame_nombre_vecino(x, i):
    return x[1][i][1]

In [79]:
def iterar_grafo(rddLista):
    nodo = rddLista.first()
    print(nodo[0])
    lista = range(0, 10)
    for i in lista:
        j = 0
        nombreVecino = dame_nombre_vecino(nodo, j)
        posibleVecino = rddLista.filter(lambda x: x[0] == nombreVecino)
        while (posibleVecino.isEmpty()):
            j = j + 1
            nombreVecino = dame_nombre_vecino(nodo, j)
            posibleVecino = rddLista.filter(lambda x: x[0] == nombreVecino)
            
        vecino = posibleVecino.first()
        print(vecino[0])
        nodo = vecino
        

In [80]:
iterar_grafo(rddListaDeAdyacencia)

Luis Emilio Sierra
2006
Dinamarca
Wikiviajes
Alemania
Cerveza
Bierpalast
Alemania
Cerveza
Bierpalast
Alemania


## Conclusión
Después de haber hecho la lista de adyacencias del grafo, se puede realizar un recorrido por sus nodos. Acá se puede observar que hay un ciclo cerrado entre Alemania, la Cerveza, y el Bierpalast, será casualidad?

## En limpio
A lo largo de la consigna se fueron descubriendo los rdds para ir verificando cuáles operaciones hacer y que todo funcione correctamente. Con esto en mente está claro que la consigna puede ser resuelta con menos líneas de código, por lo que se procede a mostrar una solución 'final', con los rdds recién leídos y cargados, saltándose los pasos intermedios de investigación de los rdds.

Las siguientes transformaciones y acciones están hechas sobre los rdds originales obtenidos al hacer la lectura de los archivos correspondientes.

Las funciones para iterar el grafo están definidas en el cuerpo de desarrollo del presente archivo. Incluyelas antes de ejecutar en limpio.

In [81]:
rddPageLinks = rddPageLinks.map(lambda x: (x[0], x[1])).filter(lambda x: ((x[0] != None) and (x[1] != None)))
rddContents = rddContents.filter(lambda x: ((x[0] != None) and (x[1] != None))).map(lambda x: (x[1], x[0]))

In [82]:
rddListaDeAdyacencia = rddContents.join(rddPageLinks).map(lambda x: (x[1][0], x[1][1])).groupBy(lambda x: x[0])\
                        .map(lambda x: (x[0], list(x[1])))
rddListaDeAdyacencia.take(5)

[('Partenón',
  [('Partenón', 'Votiva'),
   ('Partenón', 'Pronaos'),
   ('Partenón', 'Constantinopla'),
   ('Partenón', 'Atenas'),
   ('Partenón', 'Bajorrelieve'),
   ('Partenón', 'ISSN'),
   ('Partenón', 'Anastilosis'),
   ('Partenón', 'Apolo'),
   ('Partenón', 'Entablamento'),
   ('Partenón', 'ISBN'),
   ('Partenón', 'Friso'),
   ('Partenón', 'Bronce'),
   ('Partenón', 'Plutarco'),
   ('Partenón', 'Cambridge'),
   ('Partenón', 'Atenea'),
   ('Partenón', 'Olivo'),
   ('Partenón', 'Mezquita'),
   ('Partenón', 'Panateneas'),
   ('Partenón', 'Fidias'),
   ('Partenón', 'Virgilio'),
   ('Partenón', 'Paros'),
   ('Partenón', 'Zeus'),
   ('Partenón', 'Criselefantino'),
   ('Partenón', 'Wikidata'),
   ('Partenón', 'Mundo'),
   ('Partenón', 'Persia'),
   ('Partenón', 'Jerjes'),
   ('Partenón', 'Recta'),
   ('Partenón', 'Acrotera'),
   ('Partenón', 'Polis'),
   ('Partenón', 'Tridente'),
   ('Partenón', 'Delos')]),
 ('BAC', [('BAC', 'Catolicismo')]),
 ('Bomba atómica',
  [('Bomba atómica', 'Isra

In [83]:
iterar_grafo(rddListaDeAdyacencia)

Partenón
Pronaos
Fachada
Azotea
Pararrayos
Incendio
Madera
Higroscopicidad
Higroscopia
Hidroxilamina
KEGG
